# Create Python Recommender System Using Cosine Similarity
- Recommend Next Best Action for Machines that are Predicted to Fail

In [12]:
#import pandas as pd

#df = pd.read_csv('/sasinside/userdata/gegrab/resources/hmeq/machine_anomaly_remediation_history.csv',sep= ',')
# df =pd.read_csv("Data_orig/machine_anomaly.csv")

# df.S_1.max(), df.S_1.min()

# X_test2 = pd.read_csv("Data_orig/machine_anomaly_test_1.csv")

# X_test2
# df.to_csv('Data_orig/machine_anomaly_remediation_history.csv', index=False)


# gene = pd.read_csv('Data_orig/machine_anomaly_remediation_history.csv')

# gene

In [489]:
import pandas as pd
import numpy as np
import random
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder
from datetime import date
 
df =pd.read_csv("Data_orig/machine_anomaly.csv")

df.drop([287, 288, 289, 290, 291, 292, 293, 294, 295, 296,], inplace=True)
df.drop(['TARGET'], axis=1, inplace=True)

df['MACHINE_ID'] = [ random.randint(1,10000000)  for k in df.index]
#PROVIDE HISTORICAL DATES
df.insert(1, 'REPAIR_DATE',pd.date_range(start='4/1/2008', periods=len(df), freq='D'))
df.drop('SENSOR_DATE', axis=1, inplace=True)

df['REPAIR_DATE'] = df['REPAIR_DATE'].apply(lambda x: x.strftime('%Y-%m-%d'))

#df=df.sample(frac=1).reset_index(drop=True)

df['index']= df.index

def action_if(x): 
    if (x < 300):
        return 'Turn Off Machine'
    elif (x < 500):
        return 'Review Capacity'
    elif (x < 1000):
        return 'Run at 25% Capacity'
    elif (x < 1500):
        return 'Run at 50% Capacity'
    elif (x < 2000):
        return 'Run at 75% Capacity'
    elif (x < 2500):
        return 'Check Bearings'
    elif (x < 3000):
        return 'Check Belts'
    elif (x < 3500):
        return 'Check for Corrosion'
    elif (x < 4000):
        return 'Check for Misalignment'
    elif (x < 4200):
        return 'Test Electrical'
    elif (x < 4400):
        return 'Test Power Source'
    
    else:
        return 'Test HVAC'
    
action_group = df.copy()   
df['ACTION'] = action_group['index'].apply(action_if)

df=df.sample(frac=1).reset_index(drop=True)

df_history=df.drop('index', axis=1)


 
df_history.to_csv('/sasinside/userdata/gegrab/resources/hmeq/machine_anomaly_remediation_history.csv', index=False)

dfgene=df_history.drop(['MACHINE_ID', 'REPAIR_DATE', 'ACTION'], axis=1).to_numpy(dtype= 'float32')
dfgene3=df_history.drop(['MACHINE_ID', 'REPAIR_DATE', 'ACTION'], axis=1)


df.head(1)


 
#sort = df.sort_values('REPAIR_DATE', axis = 0, ascending = False)
 

,MACHINE_ID,REPAIR_DATE,S_0,S_1,S_2,S_3,S_4,S_5,S_6,S_7,...,S_132,S_133,S_134,S_135,S_136,S_137,S_138,S_139,index,ACTION
0,6687764,2015-05-26,0.921097,-0.680537,-2.390337,-3.148437,-4.021497,-3.908988,-2.698628,-1.96428,...,1.158401,1.59263,1.017008,1.16329,0.785692,-0.024698,-0.974434,-0.749608,2621,Check Belts


In [490]:
machine_index =0
cosine_sim = cosine_similarity(dfgene)
similar_machines = list(enumerate(cosine_sim[machine_index])) 
sorted_similar_machines = sorted(similar_machines, key=lambda x:x[1], reverse=True)
print(sorted_similar_machines[0:11])
     

[(0, 1.0000006), (4257, 0.98723), (2921, 0.98603064), (1867, 0.98520947), (4796, 0.984967), (3360, 0.9847487), (4712, 0.9834883), (4212, 0.9833947), (757, 0.9831398), (2784, 0.982866), (4197, 0.9823977)]


In [491]:
def get_machine_from_index2(index):
    #return df[['MACHINE_ID' ,'FAILURE']]
    return (df[df.index == index]["MACHINE_ID"].values[0], df[df.index == index]["ACTION"].values[0],
            df[df.index == index]["REPAIR_DATE"].values[0])
            #sorted_similar_machines[i][1], sorted_similar_machines[i][0])
            
 

print('Machine ID with Predicted Failure:\n',df[df.index == 0]["MACHINE_ID"].values[0]) 
print('\n')

i=0
print('"Machine IDs with Similar Anomaly Features and the Ensuing Response Action"')
print('Machine IDs + Remediation + Remediation Date:\n')
for machine in sorted_similar_machines:
    i=i+1
    
    if i > 1:
        print(get_machine_from_index2(machine[0]))  
    if i  > 10:
        break
    
        




Machine ID with Predicted Failure:
 6687764


"Machine IDs with Similar Anomaly Features and the Ensuing Response Action"
Machine IDs + Remediation + Remediation Date:

(9525394, 'Check Bearings', '2015-01-22')
(5059578, 'Run at 75% Capacity', '2013-03-14')
(1267220, 'Run at 75% Capacity', '2012-06-13')
(2275559, 'Run at 75% Capacity', '2012-11-21')
(7178472, 'Run at 25% Capacity', '2009-08-22')
(4147575, 'Check Bearings', '2014-05-05')
(6160110, 'Run at 50% Capacity', '2011-10-01')
(7648760, 'Check Belts', '2015-10-31')
(2530454, 'Check Bearings', '2014-12-15')
(2828313, 'Check Bearings', '2013-10-29')


In [492]:
df.dtypes

MACHINE_ID       int64
REPAIR_DATE     object
S_0            float64
S_1            float64
S_2            float64
                ...   
S_137          float64
S_138          float64
S_139          float64
index            int64
ACTION          object
Length: 144, dtype: object

In [493]:
def execute(MACHINE_ID, SENSOR_DATE, S_0, S_1, S_2, S_3, S_4, S_5, S_6, S_7, S_8, S_9, S_10, S_11, S_12, S_13, S_14, S_15, S_16, S_17, S_18, S_19, S_20, S_21, S_22, S_23, S_24, S_25, S_26, S_27, S_28, S_29, S_30, S_31, S_32, S_33, S_34, S_35, S_36, S_37, S_38, S_39, S_40, S_41, S_42, S_43, S_44, S_45, S_46, S_47, S_48, S_49, S_50, S_51, S_52, S_53, S_54, S_55, S_56, S_57, S_58, S_59, S_60, S_61, S_62, S_63, S_64, S_65, S_66, S_67, S_68, S_69, S_70, S_71, S_72, S_73, S_74, S_75, S_76, S_77, S_78, S_79, S_80, S_81, S_82, S_83, S_84, S_85, S_86, S_87, S_88, S_89, S_90, S_91, S_92, S_93, S_94, S_95, S_96, S_97, S_98, S_99, S_100, S_101, S_102, S_103, S_104, S_105, S_106, S_107, S_108, S_109, S_110, S_111, S_112, S_113, S_114, S_115, S_116, S_117, S_118, S_119, S_120, S_121, S_122, S_123, S_124, S_125, S_126, S_127, S_128, S_129, S_130, S_131, S_132, S_133, S_134, S_135, S_136, S_137, S_138, S_139 ):
    "Output: COSINE_1, MACHINE_ID_1, REPAIR_DATE_1, ACTION_1"
             
    import pandas as pd
    import pickle
    import numpy as np
    from sklearn.metrics.pairwise import cosine_similarity
    
    inputArray = pd.DataFrame([[MACHINE_ID, SENSOR_DATE, S_0, S_1, S_2, S_3, S_4, S_5, S_6, S_7, S_8, S_9, S_10, S_11, S_12, S_13, S_14, S_15, S_16, S_17, S_18, S_19, S_20, S_21, S_22, S_23, S_24, S_25, S_26, S_27, S_28, S_29, S_30, S_31, S_32, S_33, S_34, S_35, S_36, S_37, S_38, S_39, S_40, S_41, S_42, S_43, S_44, S_45, S_46, S_47, S_48, S_49, S_50, S_51, S_52, S_53, S_54, S_55, S_56, S_57, S_58, S_59, S_60, S_61, S_62, S_63, S_64, S_65, S_66, S_67, S_68, S_69, S_70, S_71, S_72, S_73, S_74, S_75, S_76, S_77, S_78, S_79, S_80, S_81, S_82, S_83, S_84, S_85, S_86, S_87, S_88, S_89, S_90, S_91, S_92, S_93, S_94, S_95, S_96, S_97, S_98, S_99, S_100, S_101, S_102, S_103, S_104, S_105, S_106, S_107, S_108, S_109, S_110, S_111, S_112, S_113, S_114, S_115, S_116, S_117, S_118, S_119, S_120, S_121, S_122, S_123, S_124, S_125, S_126, S_127, S_128, S_129, S_130, S_131, S_132, S_133, S_134, S_135, S_136, S_137, S_138, S_139]],
                              columns = ["MACHINE_ID", "SENSOR_DATE", "S_0", "S_1", "S_2", "S_3", "S_4", "S_5", "S_6", "S_7", "S_8", "S_9", "S_10", "S_11", "S_12", "S_13", "S_14", "S_15", "S_16", "S_17", "S_18", "S_19", "S_20", "S_21", "S_22", "S_23", "S_24", "S_25", "S_26", "S_27", "S_28", "S_29", "S_30", "S_31", "S_32", "S_33", "S_34", "S_35", "S_36", "S_37", "S_38", "S_39", "S_40", "S_41", "S_42", "S_43", "S_44", "S_45", "S_46", "S_47", "S_48", "S_49", "S_50", "S_51", "S_52", "S_53", "S_54", "S_55", "S_56", "S_57", "S_58", "S_59", "S_60", "S_61", "S_62", "S_63", "S_64", "S_65", "S_66", "S_67", "S_68", "S_69", "S_70", "S_71", "S_72", "S_73", "S_74", "S_75", "S_76", "S_77", "S_78", "S_79", "S_80", "S_81", "S_82", "S_83", "S_84", "S_85", "S_86", "S_87", "S_88", "S_89", "S_90", "S_91", "S_92", "S_93", "S_94", "S_95", "S_96", "S_97", "S_98", "S_99", "S_100", "S_101", "S_102", "S_103", "S_104", "S_105", "S_106", "S_107", "S_108", "S_109", "S_110", "S_111", "S_112", "S_113", "S_114", "S_115", "S_116", "S_117", "S_118", "S_119", "S_120", "S_121", "S_122", "S_123", "S_124", "S_125", "S_126", "S_127", "S_128", "S_129", "S_130", "S_131", "S_132", "S_133", "S_134", "S_135", "S_136", "S_137", "S_138", "S_139"]
                               )
    
    
    inputArray2 =inputArray.drop(['MACHINE_ID', 'SENSOR_DATE'], axis=1)
    inputArray2['index']= inputArray2.index
    
    df = pd.read_csv('/sasinside/userdata/gegrab/resources/hmeq/machine_anomaly_remediation_history.csv',sep= ',')
    
     
 
          
    df['index']= df.index+1
    
    df2 =df.drop(['MACHINE_ID', 'REPAIR_DATE', 'ACTION'], axis=1)
    
    
     
    combined = pd.concat([inputArray2,df2], sort=False)
    callcenter_index =0
    cosine_sim = cosine_similarity(combined) 
    similar_callcenter = list(enumerate(cosine_sim[callcenter_index])) 
    sorted_similar_callcenter = pd.DataFrame(sorted(similar_callcenter, key=lambda x:x[1], reverse=True))
    sorted_similar_callcenter.columns  = ['index', 'cosine_sim']
    
    merged = pd.merge(sorted_similar_callcenter, df,on=['index'])
    
     
    
    
    COSINE_1 = float(merged.iloc[0][1])
      
    MACHINE_ID_1 = (merged.iloc[0][2])

    REPAIR_DATE_1 = merged.iloc[0][3]

         
    ACTION_1 = merged.iloc[0][144]

     
     
   
     
   
   
    return (COSINE_1, MACHINE_ID_1, REPAIR_DATE_1, ACTION_1)

In [495]:
import pandas as pd
X_test2 = pd.read_csv("Data_orig/machine_anomaly_test_1.csv")

for i in range(10):
    print(execute(**X_test2.iloc[i]))
 



(0.8571899884581589, 6687764, '2015-05-26', 'Check Belts')
(0.866271995937594, 6687764, '2015-05-26', 'Check Belts')
(0.6757392308574384, 3924224, '2016-05-10', 'Check Belts')
(0.9317361940001353, 6687764, '2015-05-26', 'Check Belts')
(0.9295931334873656, 6687764, '2015-05-26', 'Check Belts')
(0.8649566309783638, 8944930, '2018-07-26', 'Check for Misalignment')
(0.8461638972427812, 4373215, '2020-11-30', 'Test HVAC')
(0.8315365364111457, 9672140, '2016-12-26', 'Check for Corrosion')
(0.8681750573811275, 4373215, '2020-11-30', 'Test HVAC')
(0.8607323939816349, 4373215, '2020-11-30', 'Test HVAC')
